# Exercises: Connecting to PostgreSQL with Python

There may be other ways to solve these exercises.

In [ ]:
import psycopg2

## Exercise: Create and Populate Tables

Connect to a database where you have permission to create tables.

Create three tables with appropriate columns:

* `person`: at least an ID and name, maybe other characteristics of a person
* `relationship`: links people together and labels it with a relationship type
* `relationship_type`: a table defining the allowed set of relationship types in the `relationship` table

Populate the tables with information about your friends and/or family.

Print out sentences describing the family relationships.

#### Solution

There are obviously multiple ways to do this.  Here is one.

In [ ]:
conn = psycopg2.connect(dbname="", host="", user="", password="") # fill in details
cur = conn.cursor()

In [ ]:
# create tables
cur.execute("create table person (id serial primary key, name text not null);")
cur.execute("""create table relationship_type (
               type text primary key);""")
cur.execute("""create table relationship (
            id serial primary key, 
            subject int references person(id),
            predicate int references person(id),
            relationship text references relationship_type(type));""")
conn.commit()

Populate tables.  One option is to create a dict to store the auto generated IDs for the people to use later.

In [ ]:
family = {x:None for x in ['Christina','Casey','Henry','Jessica','Denise','Bob']}
for person in family:
    # insert
    cur.execute("insert into person (name) values (%s);", [person])
    # retrieve ID
    cur.execute("select id from person where name=%s;", [person])
    family[person] = cur.fetchone()[0]
    
# commit
conn.commit()

Define relationship types.  There's no reason you have to use an underscore in the names -- you could use a space.  The use of the underscore comes from experience with categorical variables in data analysis in other contexts.

In [ ]:
for rtype in ['spouse_of','parent_of','sibling_of','child_of']:
    cur.execute("insert into relationship_type values (%s);", [rtype])

# commit
conn.commit()

In [ ]:
# not complete set of relationships, but we can do some both ways
cur.execute("""insert into relationship (subject, predicate, relationship) 
                values (%s, %s, %s);""", (family['Christina'], family['Casey'], 'spouse_of'))
cur.execute("""insert into relationship (subject, predicate, relationship) 
                values (%s, %s, %s);""", (family['Christina'], family['Henry'], 'parent_of'))
cur.execute("""insert into relationship (subject, predicate, relationship) 
                values (%s, %s, %s);""", (family['Casey'], family['Henry'], 'parent_of'))
cur.execute("""insert into relationship (subject, predicate, relationship) 
                values (%s, %s, %s);""", (family['Henry'], family['Christina'], 'child_of'))
cur.execute("""insert into relationship (subject, predicate, relationship) 
                values (%s, %s, %s);""", (family['Henry'], family['Casey'], 'child_of'))
cur.execute("""insert into relationship (subject, predicate, relationship) 
                values (%s, %s, %s);""", (family['Christina'], family['Jessica'], 'sibling_of'))
cur.execute("""insert into relationship (subject, predicate, relationship) 
                values (%s, %s, %s);""", (family['Christina'], family['Denise'], 'child_of'))
cur.execute("""insert into relationship (subject, predicate, relationship) 
                values (%s, %s, %s);""", (family['Christina'], family['Bob'], 'child_of'))
cur.execute("""insert into relationship (subject, predicate, relationship) 
                values (%s, %s, %s);""", (family['Jessica'], family['Denise'], 'child_of'))
cur.execute("""insert into relationship (subject, predicate, relationship) 
                values (%s, %s, %s);""", (family['Jessica'], family['Bob'], 'child_of'))
conn.commit()

Note that instead of manually entering each relationship both ways, you could set up [triggers](https://www.postgresql.org/docs/9.1/static/sql-createtrigger.html) in the database to do this.  This would take some work to set up (you'd need to define the opposite of each relationship type), but it's possible.

Look at results.

In [ ]:
cur.execute("""select a.name, b.name, relationship from person a, person b, relationship 
                where a.id=subject and b.id=predicate;""")
for row in cur.fetchall():
    print("{} is the {} {}.".format(row[0], row[2].replace("_", " "), row[1]))

In [ ]:
cur.close()
conn.close()